### Messing with trying to get error injection to work with .fit()

#### first try using a custom model with train step override

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import error_inject_layer
import dense_error_injection
import error_inject_optimizer
from tensorflow.python.util import deprecation
print(tf.__version__)

2.5.0


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Load Data

In [13]:
mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [4]:
mnist

<module 'tensorflow.keras.datasets.mnist' from '/home/derek/projects/ml/tf_env/lib/python3.8/site-packages/tensorflow/keras/datasets/mnist/__init__.py'>

In [5]:
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

### Preprocess Data

In [6]:
# Scale values
train_images = train_images / 255.0
test_images = test_images / 255.0

## Create Custom Model

In [43]:
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [44]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
#     tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [45]:
def step(X, y):
	# keep track of our gradients
	with tf.GradientTape() as tape:
		# make a prediction using the model and then calculate the
		# loss
		pred = model(X)
		loss = categorical_crossentropy(y, pred)
	# calculate the gradients using our tape and then update the
	# model weights
	grads = tape.gradient(loss, model.trainable_variables)
	opt.apply_gradients(zip(grads, model.trainable_variables))

In [46]:
# initialize the number of epochs to train for, batch size, and
# initial learning rate
EPOCHS = 25
BS = 32
INIT_LR = 1e-3
# load the MNIST dataset
print("[INFO] loading MNIST dataset...")
((trainX, trainY), (testX, testY)) = mnist.load_data()
# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
# trainX = np.expand_dims(trainX, axis=-1)
# testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
# one-hot encode the labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# build our model and initialize our optimizer
print("[INFO] creating model...")
# model = build_model(28, 28, 1, 10)
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)

[INFO] loading MNIST dataset...
[INFO] creating model...


In [47]:
numUpdates = int(trainX.shape[0] / BS)
print(numUpdates)

1875


In [48]:
# compute the number of batch updates per epoch
numUpdates = int(trainX.shape[0] / BS)
# loop over the number of epochs
for epoch in range(0, EPOCHS):
	# show the current epoch number
	print(f"[INFO] starting epoch {epoch}/{EPOCHS}...")
# 	sys.stdout.flush()
# 	epochStart = time.time()
	# loop over the data in batch size increments
	for i in range(0, numUpdates):
		# determine starting and ending slice indexes for the current
		# batch
		start = i * BS
		end = start + BS
		# take a step
		step(trainX[start:end], trainY[start:end])
	# show timing information for the epoch
# 	epochEnd = time.time()
# 	elapsed = (epochEnd - epochStart) / 60.0
# 	print("took {:.4} minutes".format(elapsed))

[INFO] starting epoch 0/25...
[INFO] starting epoch 1/25...
[INFO] starting epoch 2/25...
[INFO] starting epoch 3/25...
[INFO] starting epoch 4/25...
[INFO] starting epoch 5/25...
[INFO] starting epoch 6/25...
[INFO] starting epoch 7/25...
[INFO] starting epoch 8/25...
[INFO] starting epoch 9/25...
[INFO] starting epoch 10/25...
[INFO] starting epoch 11/25...
[INFO] starting epoch 12/25...
[INFO] starting epoch 13/25...
[INFO] starting epoch 14/25...
[INFO] starting epoch 15/25...
[INFO] starting epoch 16/25...
[INFO] starting epoch 17/25...
[INFO] starting epoch 18/25...
[INFO] starting epoch 19/25...
[INFO] starting epoch 20/25...
[INFO] starting epoch 21/25...
[INFO] starting epoch 22/25...
[INFO] starting epoch 23/25...
[INFO] starting epoch 24/25...


In [49]:
model.compile(optimizer=opt, loss=categorical_crossentropy,
	metrics=["acc"])
# now that the model is compiled we can compute the accuracy
(loss, acc) = model.evaluate(testX, testY)
print("[INFO] test accuracy: {:.4f}".format(acc))

313/313 [==============================] - 1s 1ms/step - loss: 0.0965 - acc: 0.9803
[INFO] test accuracy: 0.9803


### Maybe sparse categorical cross entropy was the problem with custom training

Should retry everything with categorical cross entropy